In [1150]:
!pip install --quiet virtualenv
!virtualenv /kaggle/temp_env
!source /kaggle/temp_env/bin/activate && pip install --no-cache-dir scikit-learn==1.2.2 imbalanced-learn==0.10.1


created virtual environment CPython3.11.13.final.0-64 in 310ms
  creator CPython3Posix(dest=/kaggle/temp_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: imbalanced_learn==0.10.1, joblib==1.5.2, numpy==2.3.4, pip==25.2, scikit_learn==1.2.2, scipy==1.16.2, setuptools==80.9.0, sklearn_compat==0.1.4, threadpoolctl==3.6.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
Error in sitecustomize; set PYTHONVERBOSE for traceback:
ModuleNotFoundError: No module named 'log'


In [1151]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import missingno as msno
import matplotlib.pyplot as plt  
import seaborn as sns
%matplotlib inline


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-modeling-dirty-dl-assignment1/Churn_Modelling_Dirty.csv


# Task 1: Data Loading and Exploration

**Load the data and display the first 10 rows**

In [1152]:
df=pd.read_csv('/kaggle/input/churn-modeling-dirty-dl-assignment1/Churn_Modelling_Dirty.csv')
df.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,8056,15659807,Nwachinemelu,NaN,Spain,Male,41.0,8.0,109402.13,1,1,1,66463.62,0
1,3743,15698031,Romano,587.0,Germany,Female,39.0,6.0,101851.80,2,1,0,7103.71,0
2,2700,15600103,Alexander,633.0,Germany,Female,29.0,8.0,104944.10,1,1,1,97684.46,0
3,5724,15680224,Ross,687.0,France,Female,26.0,6.0,0.00,2,1,1,32909.13,0
4,8988,15695932,Yelverton,766.0,Spain,Male,36.0,5.0,78381.13,1,0,1,153831.60,0
5,6523,15632185,Yermolayev,663.0,France,Female,42.0,1.0,82228.67,2,1,0,71359.78,0
6,3150,15723373,Page,643.0,Spain,Female,34.0,8.0,117451.47,1,1,0,65374.86,0
7,2877,15585284,Thomson,604.0,Spain,Female,35.0,7.0,147285.52,1,1,1,57807.05,0
8,5915,15726058,Cattaneo,754.0,Germany,Male,27.0,7.0,117578.35,2,0,1,87908.01,0
9,7745,15806822,Myers,739.0,France,Female,36.0,0.0,0.00,2,0,0,133465.57,0


**Get the datatype of each column**

Use appropriate pandas method to display data types

In [1153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10200 non-null  int64  
 1   CustomerId       10200 non-null  int64  
 2   Surname          10200 non-null  object 
 3   CreditScore      9469 non-null   float64
 4   Geography        10200 non-null  object 
 5   Gender           9383 non-null   object 
 6   Age              9711 non-null   float64
 7   Tenure           9453 non-null   float64
 8   Balance          9286 non-null   float64
 9   NumOfProducts    10200 non-null  int64  
 10  HasCrCard        10200 non-null  int64  
 11  IsActiveMember   10200 non-null  int64  
 12  EstimatedSalary  9552 non-null   float64
 13  Exited           10200 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 1.1+ MB


In [1154]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

Identify which columns are numerical and which are categorical

In [1155]:
numerical_features=df.select_dtypes(include='number')
categorical_features=df.select_dtypes(include='object')
numerical_features

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,8056,15659807,NaN,41.0,8.0,109402.13,1,1,1,66463.62,0
1,3743,15698031,587.0,39.0,6.0,101851.80,2,1,0,7103.71,0
2,2700,15600103,633.0,29.0,8.0,104944.10,1,1,1,97684.46,0
3,5724,15680224,687.0,26.0,6.0,0.00,2,1,1,32909.13,0
4,8988,15695932,766.0,36.0,5.0,78381.13,1,0,1,153831.60,0
...,...,...,...,...,...,...,...,...,...,...,...
10195,5735,15596647,768.0,54.0,8.0,69712.74,1,1,1,69381.05,0
10196,5192,15681075,682.0,58.0,1.0,0.00,1,1,1,706.50,0
10197,5391,15573851,735.0,38.0,1.0,0.00,3,0,0,92220.12,1
10198,861,15807663,667.0,43.0,8.0,190227.46,1,1,0,97508.04,1


In [1156]:
categorical_features

,Surname,Geography,Gender
0,Nwachinemelu,Spain,Male
1,Romano,Germany,Female
2,Alexander,Germany,Female
3,Ross,France,Female
4,Yelverton,Spain,Male
...,...,...,...
10195,Henderson,France,Male
10196,Chukwualuka,France,Female
10197,Macrossan,France,Female
10198,McGregor,France,Male


**Print the shape of the dataset**

Display number of rows and columns

In [1157]:
df.shape

(10200, 14)

Print a summary statement like: "The dataset contains X rows and Y columns"

In [1158]:
#df dataset has 10200 rows and 14 columns

**Generate descriptive statistics for numerical features**

Use .describe() for numerical columns

Include count, mean, std, min, max, and quartiles

In [1159]:
numerical_features.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10200.000000,1.020000e+04,9469.000000,9711.000000,9453.000000,9286.000000,10200.000000,10200.000000,10200.000000,9552.000000,10200.000000
mean,5005.779118,1.569105e+07,656.203717,38.881887,5.016397,88196.214094,1.530000,0.705588,0.515588,103125.446895,0.205098
std,2886.897622,7.191773e+04,94.450348,10.085472,2.891728,96522.194442,0.585039,0.455800,0.499781,69513.943008,0.403793
min,1.000000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,2505.750000,1.562871e+07,590.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,49845.020000,0.000000
50%,5007.500000,1.569073e+07,658.000000,37.000000,5.000000,99792.980000,1.000000,1.000000,1.000000,100185.240000,0.000000
75%,7505.250000,1.575348e+07,722.000000,44.000000,7.000000,129927.887500,2.000000,1.000000,1.000000,150687.745000,0.000000
max,10000.000000,1.581569e+07,850.000000,92.000000,10.000000,799812.087220,4.000000,1.000000,1.000000,597636.489250,1.000000


Provide a brief interpretation of the statistics

In [1160]:
for i in numerical_features.columns:
    col=numerical_features[i].dropna()
    mean=col.mean()
    mode=col.median()
    std=col.std()
    skewness = (mean - mode) / std
    print(i,':',skewness)

RowNumber : -0.0005961009285353658
CustomerId : 0.00440824779869784
CreditScore : -0.01901827414228075
Age : 0.18659380458504057
Tenure : 0.0056702821025577214
Balance : -0.12014610704410085
NumOfProducts : 0.9059222738064864
HasCrCard : -0.645922814992907
IsActiveMember : -0.9692471921337595
EstimatedSalary : 0.04229664967490516
Exited : 0.5079286148612623


In [1161]:
#RowNumber skewness=-0.000596 which means that RowNumber column is almost symmetric but skewed alittle to the left
#CustomerId skewness=0.00441 which means that CustomerId column is almost symmetric but skewed alittle to the right
#CreditScore skewness=-0.01902 which means that CreditScore column is almost symmetric but skewed alittle to the left
#Age skewness=0.1866 which means that Age column is almost symmetric but skewed alittle to the right
#Tenure skewness=0.00567 which means that Tenure column is almost symmetric but skewed alittle to the right
#Balance skewness=-0.1201 which means that Balance column is skewed alittle to the left
#NumOfProducts skewness=0.9059 which means that NumOfProducts column is highly skewed to the right
#HasCrCard skewness=-0.646 which means that HasCrCard  column is moderatly skewed to the left
#IsActiveMember skewness=-0.9692 which means that IsActiveMember column is highly skewed to the left
#EstimatedSalary skewness= 0.0422 which means that EstimatedSalary is alittle skewed to the right
#Exited skewness= 0.5079 which means that Exited  column is moderatly skewed to the right

# Task 2: Dataset Quality Assessment

**2.1 Missing Values Analysis**

**Check for the percentage of missing values in each column**

Missing Values Analysis

In [1162]:
df.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore        731
Geography            0
Gender             817
Age                489
Tenure             747
Balance            914
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary    648
Exited               0
dtype: int64

In [ ]:
msno.matrix(df)
plt.show()

Check for the percentage of missing values in each column

In [ ]:
df_null=pd.DataFrame(df.isnull().sum()/10200*100)
df_null=df_null.apply(lambda x:round(x,2).astype(np.float32).map('{:.3f}%'.format))
df_null=df_null.rename(columns={0:'Percentage'})
df_null

Create a summary table showing: Column Name | Missing Count | Missing Percentage

In [ ]:
df_summary_table=pd.DataFrame(df.isnull().sum()/10200*100)
df_summary_table=df_summary_table.apply(lambda x:round(x,2).astype(np.float32).map('{:.3f}%'.format))
df_summary_table=df_summary_table.rename(columns={0:'Percentage'})
df_summary_table['Count']=df.isnull().sum()
df_summary_table.sort_values(by='Percentage',ascending=False)

**Visualize missing values**

Create a bar plot showing percentage of missing values per column

In [ ]:
df_summary_table.reset_index(inplace=True)

In [ ]:
df_summary_table.rename(columns={'index':'Column Name'},inplace=True)
df_summary_table

In [ ]:
Percentage=df_summary_table['Percentage'].str.replace('%','',regex=False).astype(np.float32)

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['font.size'] =10
plt.figure(figsize=(20,20))
sns.barplot(data=df_summary_table, x='Column Name', y=Percentage)
plt.yticks(np.arange(0,10.5,0.5))
plt.show()

In [ ]:
plt.rcParams['font.size'] =8
plt.figure(figsize=(9,8))
corr_matrix=numerical_features.corr(method='pearson')   
sns.heatmap(corr_matrix, annot=True, fmt=".2f")

**Determine the missing values mechanism and handling method**

In [ ]:
#column with missing values
#MCAR (Missing Completely At Random): Missing values are random and unrelated to any variable

In [ ]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)


In [ ]:
df

**2.2 Duplicate Detection**

Count total number of duplicate rows

In [ ]:
df.duplicated().sum()

Display a few examples of duplicates

In [ ]:
df[df.duplicated()==True]

Decide whether to keep or remove duplicates (justify your decision)

In [ ]:
#I think I will remove duplicates as they make data more biased to some values in a wrong way

In [ ]:
df.drop_duplicates(inplace=True,keep='first')

In [ ]:
df.duplicated().sum()

If removing, show the shape before and after

In [ ]:
#shape of data before removing duplicates 10200 rows x 11 columns
df.shape
#shape of data after removing duplicates 10082 rows x 11 columns


**2.3 Outliers and Noisy Data Detection**

Detect outliers and noisy data through visualizations

**a) Box plots for numerical features**

Create box plots for: CreditScore, Age, Balance, EstimatedSalary

In [ ]:
plt.rcParams['font.size'] =8
sns.boxplot(data=df,x='Age')

In [ ]:
plt.rcParams['font.size'] =8
sns.boxplot(data=df,x='Balance',color='yellow')

In [ ]:
plt.rcParams['font.size'] =8
sns.boxplot(data=df,x='EstimatedSalary',color='red')

Bivariate analysis: Like Age vs Balance box plot, Use color coding by 'Exited' to see patterns.

In [ ]:
print(df['Age'].max(),df['Age'].min())

In [ ]:
plt.rcParams['font.size'] = 40  
plt.figure(figsize=(80,70))
sns.boxplot(data=df,x='Age',y='Balance',hue='Exited')
plt.xticks(np.arange(0,93,2))
plt.legend()

In [ ]:
plt.rcParams['font.size'] = 40  # General font size
plt.figure(figsize=(80,70))
sns.boxplot(data=df,x='Age',y='EstimatedSalary',hue='Exited')
plt.xticks(np.arange(0,93,2))
plt.legend()

Identify potential outliers also using IQR method

In [ ]:
Q1_A=df['Age'].quantile(0.25)
Q3_A=df['Age'].quantile(0.75)
IQR_A=Q3_A-Q1_A
lower_bound_A=Q1_A-1.5*IQR_A
upper_bound_A=Q3_A+1.5*IQR_A
print(f'Age_lower_bound:{lower_bound_A}  ,Age_upper_bound:{upper_bound_A} ')


In [ ]:
Q1_E=df['EstimatedSalary'].quantile(0.25)
Q3_E=df['EstimatedSalary'].quantile(0.75)
IQR_E=Q3_E-Q1_E
upper_bound_E=Q3_E+1.5*IQR_E
lower_bound_E=Q1_E-1.5*IQR_E
print(f'EstimatedSalary_lower_bound:{lower_bound_E}  ,EstimatedSalary_upper_bound:{upper_bound_E} ')



In [ ]:
Q1_B=df['Balance'].quantile(0.25)
Q3_B=df['Balance'].quantile(0.75)
IQR_B=Q3_B-Q1_B
lower_bound_B=Q1_B-1.5*IQR_B
upper_bound_B=Q3_B+1.5*IQR_B
print(f'Balance_lower_bound:{lower_bound_B}  ,Balance_upper_bound:{upper_bound_B} ')

**b) Distribution plots**

Create histograms or KDE plots for key numerical features

In [ ]:
#'CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary'
plt.rcParams['font.size'] =7
fig,axes=plt.subplots(13,2,figsize=(10,40))
axes = axes.flatten()  

cols=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
for i,col in enumerate(cols):
    sns.histplot(df[col],ax=axes[i],kde=True)
    axes[i].set_title(col)
for j in range(len(cols), len(axes)):
    axes[j].set_visible(False)
plt.tight_layout()
plt.show()

Create scatter plots showing relationships between variables:   Tenure vs NumOfProducts

In [ ]:
sns.scatterplot(data=df,x='Tenure',y='NumOfProducts',hue='Exited')
#here I can notice a pattern which is whenever NumOfProducts=4 the exited value = true

# Task 3: Comprehensive Exploratory Data Analysis

**3.1 Target Variable Analysis**

**1. Analyze the distribution of the target variable 'Exited'**

Count and percentage of each class (0: Not Churned, 1: Churned)

In [ ]:
df_exited=df['Exited'].value_counts()
df_exited=df_exited.reset_index()
df_exited

Create a bar plot showing the distribution

In [ ]:
sns.barplot(data=df_exited,x='Exited',y='count')

Create a pie chart showing the proportion

In [ ]:
colors = sns.color_palette('bright')
explode=[0,0.1]
plt.pie(data=df_exited,x=df_exited['count'],labels=['Not-Exited','Exited'], autopct='%.0f%%',colors=colors,explode=explode)

**2. Detect class imbalance**

In [ ]:
!pip show scikit-learn


In [ ]:
!pip show imbalanced-learn

In [ ]:
import sys
sys.path.insert(0, "/kaggle/temp_env/lib/python3.11/site-packages")

import sklearn
print(sklearn.__version__)

 Calculate the imbalance ratio

Determine if the dataset is balanced, slightly imbalanced, or highly imbalanced

In [ ]:
class_imbalance_ratio=df_exited['count'][0]/df_exited['count'][1]
class_imbalance_ratio
#moderately imbalanced dataset

**3.2 Univariate Analysis**

**Analyze individual feature distributions**

**For Categorical variables (Geography, Gender, HasCrCard, IsActiveMember)**

Count plots showing distribution of each category

In [ ]:
df_geography=df['Geography'].value_counts()
df_geography=df_geography.reset_index()
df_geography


In [ ]:
df_geography['percentage']=round(df_geography['count']/df_geography['count'].sum()*100,3).astype(str) + '%'
df_geography['percentage']

In [ ]:
sns.countplot(data=df, x='Geography')


In [ ]:
sns.countplot(data=df, x='Gender')


In [ ]:
df_Gender=df['Gender'].value_counts()
df_Gender=df_Gender.reset_index()
df_Gender['percentage']=round(df_Gender['count']/df_Gender['count'].sum()*100,3).astype(str) + '%'
df_Gender

In [ ]:
sns.countplot(data=df, x='HasCrCard')


In [ ]:
df_HasCrCard=df['HasCrCard'].value_counts()
df_HasCrCard=df_HasCrCard.reset_index()
df_HasCrCard['percentage']=round(df_HasCrCard['count']/df_HasCrCard['count'].sum()*100,3).astype(str) + '%'
df_HasCrCard

In [ ]:
sns.countplot(data=df, x='IsActiveMember')


In [ ]:
df_IsActiveMember=df['IsActiveMember'].value_counts()
df_IsActiveMember=df_IsActiveMember.reset_index()
df_IsActiveMember['percentage']=round(df_IsActiveMember['count']/df_IsActiveMember['count'].sum()*100,3).astype(str) + '%'
df_IsActiveMember

**For Numerical variables (CreditScore, Age, Tenure, Balance, NumOfProducts, EstimatedSalary)**

Histograms with KDE overlay

In [ ]:
#'CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary'
plt.rcParams['font.size'] =7
fig,axes=plt.subplots(13,2,figsize=(10,40))
axes = axes.flatten()  

cols=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
for i,col in enumerate(cols):
    sns.histplot(df[col],ax=axes[i],kde=True)
    axes[i].set_title(col)
for j in range(len(cols), len(axes)):
    axes[j].set_visible(False)
plt.tight_layout()
plt.show()

Statistical summary (mean, median, skewness, kurtosis)

In [ ]:
df[['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']].describe()


In [ ]:
#CreditScore skewness=-0.01902 which means that CreditScore column is almost symmetric but skewed alittle to the left
#Age skewness=0.1866 which means that Age column is almost symmetric but skewed alittle to the right
#Tenure skewness=0.00567 which means that Tenure column is almost symmetric but skewed alittle to the right
#Balance skewness=-0.1201 which means that Balance column is skewed alittle to the left
#NumOfProducts skewness=0.9059 which means that NumOfProducts column is highly skewed to the right
#EstimatedSalary skewness= 0.0422 which means that EstimatedSalary is alittle skewed to the right

Identify which features have normal distribution vs. skewed distribution

In [ ]:
#CreditScore --> normal distribution
#Age --> skewed distribution
#Tenure --> normal distribution
#Balance --> skewed distribution
#NumOfProducts --> neither normal distribution nor skewed distribution
#EstimatedSalary  --> skewed distribution

**3.3 Bivariate Analysis with Target**

**4. Analyze relationship between features and target**

**a) Categorical features vs Target (3 points)**

Create grouped bar plots showing churn rate by

In [ ]:
Exited_grouped_Geo=df.groupby('Geography')['Exited'].mean()
Exited_grouped_Geo=Exited_grouped_Geo.reset_index()
Exited_grouped_Geo

In [ ]:
sns.barplot(data=Exited_grouped_Geo,x='Geography',y='Exited')

In [ ]:
Exited_grouped_Gen=df.groupby('Gender')['Exited'].mean()
Exited_grouped_Gen=Exited_grouped_Gen.reset_index()
Exited_grouped_Gen

In [ ]:
sns.barplot(data=Exited_grouped_Gen,x='Gender',y='Exited')

In [ ]:
Exited_grouped_Cr=df.groupby('HasCrCard')['Exited'].mean()
Exited_grouped_Cr=Exited_grouped_Cr.reset_index()
Exited_grouped_Cr

In [ ]:
sns.barplot(data=Exited_grouped_Cr,x='HasCrCard',y='Exited')

In [ ]:
Exited_grouped_AM=df.groupby('IsActiveMember')['Exited'].mean()
Exited_grouped_AM=Exited_grouped_AM.reset_index()
Exited_grouped_AM

In [ ]:
sns.barplot(data=Exited_grouped_AM,x='IsActiveMember',y='Exited')

Identify which categories have higher churn rates

In [ ]:
#Female, Germany, Italy ,not active member, doesn't have credit card those are categories which have higher rates

**b) Numerical features vs Target**

Create box plots or violin plots comparing feature distributions between churned and non-churned 
customers

In [ ]:
#Age distribution by Exited
Age_grouped_Exited=df.groupby('Age')['Exited'].count()
Age_grouped_Exited=Age_grouped_Exited.reset_index()
Age_grouped_Exited

In [ ]:
sns.boxplot(data=Age_grouped_Exited,x='Age')

In [ ]:
#Age distribution by Exited
Balance_grouped_Exited=df.groupby('Balance')['Exited'].count()
Balance_grouped_Exited=Balance_grouped_Exited.reset_index()
Balance_grouped_Exited

In [ ]:
sns.boxplot(data=Balance_grouped_Exited,x='Balance')

In [ ]:
#Age distribution by Exited
CreditScore_grouped_Exited=df.groupby('CreditScore')['Exited'].count()
CreditScore_grouped_Exited=CreditScore_grouped_Exited.reset_index()
CreditScore_grouped_Exited

In [ ]:
sns.boxplot(data=CreditScore_grouped_Exited,x='CreditScore')

In [ ]:
#Age distribution by Exited
NumofProducts_grouped_Exited=df.groupby('NumOfProducts')['Exited'].count()
NumofProducts_grouped_Exited=NumofProducts_grouped_Exited.reset_index()
NumofProducts_grouped_Exited

In [ ]:
sns.boxplot(data=NumofProducts_grouped_Exited,x='NumOfProducts')

**3.4 Multivariate Analysis**

**Explore relationships between multiple variables**

**Correlation analysis**

Create a correlation heatmap for all numerical features

In [ ]:
plt.rcParams['font.size'] =8
plt.figure(figsize=(9,8))
corr_matrix=numerical_features[['Age','Balance','CreditScore','NumOfProducts']].corr(method='pearson')   
sns.heatmap(corr_matrix, annot=True, fmt=".2f")

Identify highly correlated features (correlation > 0.7 or < -0.7)

In [ ]:
#None

**Grouped analysis**

Analyze churn rate by Geography and Gender (crosstab with heatmap)

In [ ]:
ct = pd.crosstab(
    [df['Geography'], df['Gender']],
    df['Exited'],
    normalize='index'
) * 100



In [ ]:
ct.columns = ['Not Churned (%)', 'Churned (%)']

# plot heatmap for churned percentage
sns.heatmap(
    ct[['Churned (%)']],
    annot=True,
    fmt='.1f',
    cmap='Blues'
)
plt.title('Churn Rate by Geography and Gender')
plt.ylabel('Geography, Gender')
plt.show()

Analyze churn rate by Age groups and NumOfProducts

In [ ]:
ct = pd.crosstab(
    df['Age'],
    df['NumOfProducts'],
    values=df['Exited'],
    aggfunc='mean'
) * 100
ct

**Pair plots (optional but recommended)**

Create pair plots for key numerical features colored by target

In [ ]:
sns.pairplot(df[['Age', 'EstimatedSalary', 'NumOfProducts', 'CreditScore', 'Balance','Exited']],hue='Exited')
plt.tight_layout()

# Task 4: Data Preprocessing

**4.1 Handle Missing Values**

**Implement the missing value handling strategy from Task 2**

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
numerical_features=df[['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']]
numerical_imputer=IterativeImputer(estimator=DecisionTreeRegressor(),max_iter=10,random_state=0)
imputed_numerical_features=numerical_imputer.fit_transform(numerical_features)
imputed_numerical_features=pd.DataFrame(imputed_numerical_features)
imputed_numerical_features

In [ ]:
imputed_numerical_features=imputed_numerical_features.rename(columns={0:'CreditScore',1:'Age',2:'Tenure',3:'Balance',4:'NumOfProducts',5:'EstimatedSalary'}, errors="raise")
imputed_numerical_features

In [ ]:
imputed_numerical_features.isnull().sum()

**4.2 Handle Outliers**

**2. Handle outliers detected in Task 2**

In [ ]:
imputed_numerical_features['Age']=np.where(imputed_numerical_features['Age']>upper_bound_A,upper_bound_A,imputed_numerical_features['Age'])
imputed_numerical_features['Age']=np.where(imputed_numerical_features['Age']<lower_bound_A,lower_bound_A,imputed_numerical_features['Age'])
imputed_numerical_features['Age']

In [ ]:
imputed_numerical_features['EstimatedSalary']=np.where(imputed_numerical_features['EstimatedSalary']>upper_bound_E,upper_bound_E,imputed_numerical_features['EstimatedSalary'])
imputed_numerical_features['EstimatedSalary']=np.where(imputed_numerical_features['EstimatedSalary']<lower_bound_E,lower_bound_E,imputed_numerical_features['EstimatedSalary'])
imputed_numerical_features['EstimatedSalary']


In [ ]:
imputed_numerical_features['Balance']=np.where(imputed_numerical_features['Balance']<lower_bound_B,lower_bound_B,imputed_numerical_features['Balance'])
imputed_numerical_features['Balance']=np.where(imputed_numerical_features['Balance']>upper_bound_B,upper_bound_B,imputed_numerical_features['Balance'])
imputed_numerical_features['Balance']

In [ ]:
imputed_numerical_features

**4.3 Encode Categorical Variables**

In [ ]:
df.drop(columns=['CreditScore','Age','Tenure','Balance','EstimatedSalary','NumOfProducts'],inplace=True)

In [ ]:
df

In [ ]:
df['Geography'].value_counts()

In [ ]:
df = df[~df['Geography'].isin(['USA', 'Canada', 'Unknown', 'Italy', 'UK'])]
df['Geography'].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
OHE=OneHotEncoder(sparse_output=False)
Geo_encoded = OHE.fit_transform(df[['Geography']])
geo_df = pd.DataFrame(Geo_encoded, columns=['France','Germany','Spain'])

In [ ]:
geo_df

In [ ]:
Gender_encoded = OHE.fit_transform(df[['Gender']])
Gender_df = pd.DataFrame(Geo_encoded, columns=OHE.get_feature_names_out(['Gender']))

In [ ]:
Gender_df=Gender_df.drop(columns='Gender_nan')

In [ ]:
Gender_df=Gender_df.rename(columns={'Gender_Female':'Female','Gender_Male':'Male'})
Gender_df

In [ ]:
merged_df = pd.merge(Gender_df, geo_df, left_index=True, right_index=True)


In [ ]:
catigrocal_features_handeled=pd.merge(merged_df,df[['HasCrCard','IsActiveMember','Exited']], left_index=True, right_index=True)

In [ ]:
catigrocal_features_handeled

In [ ]:
new_df=pd.merge(imputed_numerical_features,catigrocal_features_handeled, left_index=True, right_index=True)

In [ ]:
new_df

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df.shape

**4.5 Train-Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
X=new_df[['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary','Female','Male','France','Germany','Spain','HasCrCard','IsActiveMember']]
y=new_df['Exited']

In [ ]:
X

In [ ]:
y

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2,stratify=y)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_test.value_counts()

In [ ]:
X_test.value_counts()

**4.4 Feature Scaling**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_train_data= scaler.fit_transform(X_train,y_train)
scaled_X_test=scaler.transform(X_test)    

In [ ]:
scaled_train_data.shape

In [ ]:
scaled_X_test

In [ ]:
#Explain why scaling is important for neural networks
'''Scaling importance for neural networks:

Neural networks are sensitive to input magnitude because weights are updated by gradient descent.

If features differ widely in scale, large-valued features dominate gradients, slowing or destabilizing training.

Scaling keeps gradients balanced and speeds convergence.
'''
#Explain why we use StandardScaler vs MinMaxScaler
'''
StandardScaler: centers data (mean = 0, std = 1).
Use when features roughly follow a normal distribution.


MinMaxScaler: rescales to [0, 1] (or custom range).
Use when all features must fit a bounded interval or distributions are not Gaussian.
'''
#CRITICAL: Only fit the scaler on training data, then transform both train and test to avoid data leakage
'''
Avoiding data leakage:
Always fit the scaler on training data only, then apply the same transformation to test data.
'''

# Task 5: Build Baseline Model 

Implement the architecture (Add hidden layer – Use L1, L2, Droupout layer, non-linear activation function)

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras import Sequential,regularizers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import  CSVLogger, EarlyStopping


model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],),
          kernel_regularizer=regularizers.l2(0.001)),  
    Dropout(0.3),
    Dense(32, activation='relu',
          kernel_regularizer=regularizers.l1(0.001)),  
    Dense(1, activation='sigmoid')
])


Display model summary showing parameters

In [ ]:
model.summary()

**Model Compilation**

In [ ]:
'''
• Use Adam optimizer with default learning rate (0.001)
• Use binary crossentropy loss
• Include accuracy, precision, recall, and AUC metrics
• Explain why we use binary crossentropy for this problem
'''
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy','precision','recall','auc'])

**Model Training**

In [ ]:
'''
• Train for maximum 100 epochs
• Use batch size of 32
• Use 20% of training data for validation
• Implement early stopping (patience=10, monitor='val_loss')
• Record training history
'''


csv_logger = CSVLogger('training_log.csv', append=True)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2,callbacks=[early_stop,csv_logger], verbose=1)



**Model Evaluation**

In [ ]:
loss, acc, precision, recall, auc = model.evaluate(X_test, y_test, verbose=1)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {acc:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test AUC: {auc:.4f}')
